In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
import gc
import xgboost as xgb
import math
from joblib import Parallel, delayed

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import time
import warnings

warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
def correlation(df, threshold=0.98):
    """
    特征相关性计算
    @param df: 
    @param threshold: 
    @return: 
    """
    col_corr = set()
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colName = corr_matrix.columns[i]
                col_corr.add(colName)
    return col_corr

In [3]:
# train_df = pd.read_hdf('../input/train.h5')
test_df = pd.read_hdf('../input/test.h5')
sub = pd.DataFrame(test_df['time'])
gc.collect()

11

In [4]:
# train_df = train_df[train_df['temperature'].notnull()]
# train_df = train_df.fillna(method='bfill')
# test_df = test_df.fillna(method='bfill')
# gc.collect()

In [5]:
# train_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
#                     'indoorHum', 'indoorAtmo', 'temperature']
test_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                   'indoorHum', 'indoorAtmo']

In [6]:
# print('train_df.shape: ', train_df.shape)
# train_df = train_df.loc[(train_df['outdoorTemp'] >= test_df['outdoorTemp'].min()) & (train_df['outdoorTemp'] <= test_df['outdoorTemp'].max())]
# print('处理后 train_df.shape: ', train_df.shape)

In [7]:
# y_train = train_df['temperature'] - train_df['outdoorTemp']
# y_train.to_hdf('../input/y_train.h5', 'df')

In [8]:
# del train_df, test_df
# gc.collect()

In [9]:
x_train = pd.read_hdf('../input/train_features_corr.h5')
x_test = pd.read_hdf('../input/test_features_corr.h5')
y_train = pd.read_hdf('../input/y_train.h5')

In [10]:
col_corr = correlation(x_train, 0.95)
print(col_corr)

{'indoorHum_MD_outdoorHum_max_multyply', 'MD_outdoorHum_diff_sum_MDH_outdoorTemp_diff_sum_multyply', 'MDH_outdoorAtmo_mean_MDH_outdoorHum_diff_sum_subtract', 'MD_outdoorAtmo_diff_sum_MDH_indoorHum_diff_sum_subtract', 'outdoorHum_50_bin_outdoorAtmo_sum', 'MD_outdoorHum_max_MDH_indoorHum_max_multyply', 'MD_outdoorHum_diff_sum_MDH_indoorHum_diff_sum_ratio', 'MD_outdoorTemp_sum_MDH_outdoorAtmo_diff_mean_ratio', 'MDH_indoorHum_diff_sum_MD_outdoorAtmo_diff_sum_multyply', 'MD_outdoorAtmo_min_MDH_outdoorTemp_diff_sum_multyply', 'outdoorAtmo_MDH_indoorHum_max_multyply', 'outdoorAtmo_MD_indoorHum_medi_multyply', 'outdoorTemp_MDH_outdoorTemp_mean_multyply', 'MDH_indoorAtmo_diff_mean_MDH_outdoorAtmo_diff_mean_multyply', 'MDH_outdoorAtmo_min_MDH_outdoorAtmo_diff_sum_ratio', 'MDH_outdoorTemp_diff_sum_MDH_indoorHum_diff_sum_multyply', 'MDH_indoorHum_diff_sum_MD_outdoorAtmo_min_multyply', 'MD_outdoorHum_diff_max_MDH_outdoorTemp_diff_sum_multyply', 'MDH_outdoorHum_medi_MDH_indoorHum_medi_subtract', 'in

In [11]:
x_train.drop(list(col_corr), axis=1, inplace=True)
x_test.drop(list(col_corr), axis=1, inplace=True)
gc.collect()

20

In [12]:
x_train.to_hdf('../input/train_features_corr_95.h5', 'df')
x_test.to_hdf('../input/test_features_corr_95.h5', 'df')